In [6]:
#!/usr/bin/env python
# coding: utf-8

import os
import numpy as np
from pathlib import Path as P

# Vanilla PyTorch
import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader, random_split

# Torchvision for CV
from torchvision.datasets import MNIST
from torchvision import transforms

# PyTorch Lightning
import lightning.pytorch as pl
import torchmetrics

# Logging
from lightning.pytorch.loggers import CSVLogger

## DATA

In [2]:
# Height -> AHN

import open3d as o3d
import numpy as np
import laspy as lp
import os 
# os.chdir(r'/tudelft.net/staff-umbrella/EDT Veluwe/testbed/AHN/')
os.chdir(r'/home/mahmoudahmed/Downloads')

# tr_pcd = lp.read("C_27DN1.LAZ")
tr_pcd = lp.read("33CN2_13.LAZ")

print([dimension.name for dimension in tr_pcd.point_format.dimensions])
# print(np.max(tr_pcd.green))
points = np.vstack((tr_pcd.x, tr_pcd.y, tr_pcd.z)).transpose()
colors = np.vstack((tr_pcd.red, tr_pcd.green, tr_pcd.blue)).transpose()

point_clouds = o3d.geometry.PointCloud()
point_clouds.points = o3d.utility.Vector3dVector(points)
point_clouds.colors = o3d.utility.Vector3dVector(colors/255) 

o3d.visualization.draw_geometries([point_clouds])

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'nir', 'Amplitude', 'Reflectance', 'Deviation']


In [8]:
# RGB -> Airal 

from qgis.core import QgsRasterLayer
from qgis.core import QgsProject  
from PyQt5.QtCore import QFileInfo
import os
os.chdir(r'/home/mahmoudahmed/Downloads/')
file = '2023_182000_461000_RGB_hrl.ecw'
fileInfo = QFileInfo(file)
baseName = fileInfo.baseName()
rlayer = QgsRasterLayer(file, baseName)
if not rlayer.isValid():
    print ("Layer failed to load again!")
else:
    print (rlayer.renderer())
    print (rlayer.renderer().type())
    if hasattr(rlayer, "setCacheImage"):
        rlayer.setCacheImage(None)
    rlayer.triggerRepaint()

# Add the layer to the Layers panel
# QgsMapLayerRegistry.instance().addMapLayer(rlayer)
    QgsProject.instance().addMapLayer(rlayer)

Layer failed to load again!


In [10]:
from osgeo import gdal
print(gdal.GetDriverByName('ECW'))

None


In [ ]:
# remove slow mirror from list of MNIST mirrors
MNIST.mirrors = [
    mirror for mirror in MNIST.mirrors if not mirror.startswith("http://yann.lecun.com")
]
pyqt

class LitMLP(pl.LightningModule):

    def __init__(self, in_dims, n_classes=10, n_layer_1=128, n_layer_2=256, lr=1e-3):
        super().__init__()

        # we flatten the input Tensors and pass them through an MLP
        self.layer_1 = nn.Linear(np.prod(in_dims), n_layer_1)
        self.layer_2 = nn.Linear(n_layer_1, n_layer_2)
        self.layer_3 = nn.Linear(n_layer_2, n_classes)

        # log hyperparameters
        self.save_hyperparameters()

        # compute the accuracy -- no need to roll your own!
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.valid_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)

    def forward(self, x):
        """
        Defines a forward pass using the Stem-Learner-Task
        design pattern from Deep Learning Design Patterns:
        https://www.manning.com/books/deep-learning-design-patterns
        """
        batch_size, *dims = x.size()

        # stem: flatten
        x = x.view(batch_size, -1)


        # task: compute class logits
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)

        return x

    # convenient method to get the loss on a batch
    def loss(self, xs, ys):
        logits = self(xs)  # this calls self.forward
        loss = F.nll_loss(logits, ys)
        return logits, loss

    def training_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)

        # logging metrics we calculated by hand
        self.log("train/loss", loss, on_epoch=True)
        # logging a pl.Metric
        self.train_acc(preds, ys)
        self.log("train/acc", self.train_acc, on_epoch=True)

        return loss

        # task: compute class logits
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)

        return x

    # convenient method to get the loss on a batch
    def loss(self, xs, ys):
        logits = self(xs)  # this calls self.forward
        loss = F.nll_loss(logits, ys)
        return logits, loss

    def training_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)

        # logging metrics we calculated by hand
        self.log("train/loss", loss, on_epoch=True)
        # logging a pl.Metric
        self.train_acc(preds, ys)
        self.log("train/acc", self.train_acc, on_epoch=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams["lr"])

    def test_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)

        self.test_acc(preds, ys)
        self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
        self.log("test/acc_epoch", self.test_acc, on_step=False, on_epoch=True)

    def on_test_epoch_end(self):
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = "output/model_final.onnx"
        self.to_onnx(model_filename, dummy_input, export_params=True)

    def on_validation_epoch_start(self):
        self.validation_step_outputs = []

    def validation_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)
        self.valid_acc(preds, ys)

        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        self.log("valid/acc_epoch", self.valid_acc)

        self.validation_step_outputs.append(logits)

        return logits

    def on_validation_epoch_end(self):
        validation_step_outputs = self.validation_step_outputs
        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        # Log metrics to CSV file
        self.logger.log_metrics(
            {
                "valid/logits": flattened_logits.mean().item(),
                "global_step": self.global_step,
            }
        )


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams["lr"])

    def test_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)lf.log("test/acc_epoch", self.test_acc, on_step=False, on_epoch=True)

    def on_test_epoch_end(self):
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = "output/model_final.onnx"
        self.to_onnx(model_filename, dummy_input, export_params=True)

    def on_validation_epoch_start(self):
        self.validation_step_outputs = []

    def validation_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)
        self.valid_acc(preds, ys)

        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        self.log("valid/acc_epoch", self.valid_acc)

        self.validation_step_outputs.append(logits)

        return logits

    def on_validation_epoch_end(self):
        validation_step_outputs = self.validation_step_outputs
        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        # Log metrics to CSV file
        self.logger.log_metrics(
            {
                "valid/logits": flattened_logits.mean().item(),
                "global_step": self.global_step,
            }
        )

        self.test_acc(preds, ys)
        self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
        self.log("test/acc_epoch", self.test_acc, on_step=False, on_epoch=True)

    def on_test_epoch_end(self):
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = "output/model_final.onnx"
        self.to_onnx(model_filename, dummy_input, export_params=True)

    def on_validation_epoch_start(self):
        self.validation_step_outputs = []

    def validation_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)
        self.valid_acc(preds, ys)

        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        self.log("valid/acc_epoch", self.valid_acc)

        self.validation_step_outputs.append(logits)

        return logits

    def on_validation_epoch_end(self):
        validation_step_outputs = self.validation_step_outputs
        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        # Log metrics to CSV file
        self.logger.log_metrics(
            {
                "valid/logits": flattened_logits.mean().item(),
                "global_step": self.global_step,
            }
        )

   # learner: two fully-connected layers
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))

        # task: compute class logits
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)

        return x
lf.log("test/acc_epoch", self.test_acc, on_step=False, on_epoch=True)

    def on_test_epoch_end(self):
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = "output/model_final.onnx"
        self.to_onnx(model_filename, dummy_input, export_params=True)

    def on_validation_epoch_start(self):
        self.validation_step_outputs = []

    def validation_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)
        self.valid_acc(preds, ys)

        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        self.log("valid/acc_epoch", self.valid_acc)

        self.validation_step_outputs.append(logits)

        return logits

    def on_validation_epoch_end(self):
        validation_step_outputs = self.validation_step_outputs
        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        # Log metrics to CSV file
        self.logger.log_metrics(
            {
                "valid/logits": flattened_logits.mean().item(),
                "global_step": self.global_step,
            }
        )
    # convenient method to get the loss on a batch
    def loss(self, xs, ys):
        logits = self(xs)  # this calls self.forward
        loss = F.nll_loss(logits, ys)
        return logits, loss

    def training_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)

        # logging metrics we calculated by hand
        self.log("train/loss", loss, on_epoch=True)
        # logging a pl.Metric
        self.train_acc(preds, ys)
        self.log("train/acc", self.train_acc, on_epoch=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams["lr"])

    def test_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)

        self.test_acc(preds, ys)
        self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
        self.log("test/acc_epoch", self.test_acc, on_step=False, on_epoch=True)

    def on_test_epoch_end(self):
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = "output/model_final.onnx"
        self.to_onnx(model_filename, dummy_input, export_params=True)

    def on_validation_epoch_start(self):
        self.validation_step_outputs = []

    def validation_step(self, batch, batch_idx):
        xs, ys = batch
        logits, loss = self.loss(xs, ys)
        preds = torch.argmax(logits, 1)
        self.valid_acc(preds, ys)

        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        self.log("valid/acc_epoch", self.valid_acc)

        self.validation_step_outputs.append(logits)

        return logits

    def on_validation_epoch_end(self):
        validation_step_outputs = self.validation_step_outputs
        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        # Log metrics to CSV file
        self.logger.log_metrics(
            {
                "valid/logits": flattened_logits.mean().item(),
                "global_step": self.global_step,
            }
        )
